# Functions

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Teoria
###################################################################################################################
def plot_curva_demanda_elasticidade_exemplo():
    import numpy as np
    import matplotlib.pyplot as plt
    # define a curva de demanda P = 4 - 2Q
    Q = np.linspace(0, 2, 200)
    P = 4 - 2 * Q
    # define o ponto de elasticidade unitária (ε = -1)
    Q_star = 1.0
    P_star = 4 - 2 * Q_star
    # cria a figura e o eixo
    fig, ax = plt.subplots(figsize=(8, 6))
    # plota a curva de demanda
    ax.plot(Q, P)
    # marca o ponto de elasticidade unitária
    ax.scatter([Q_star], [P_star], zorder=3)
    # desenha linhas tracejadas até os eixos
    ax.axvline(Q_star, ymax=P_star / P.max(), linestyle="--")
    ax.axhline(P_star, xmax=Q_star / Q.max(), linestyle="--")
    # anota o ponto crítico com leve deslocamento
    ax.text(Q_star + 0.04, P_star - 0.25, "(1, 2)", fontsize=10)
    # anotações das regiões de elasticidade
    ax.text(0.15, 3.2, "ε ∈ (-∞, -1)\nelástica", ha="left", va="center")
    ax.text(0.15, 2.15, "ε = -1\nelasticidade unitária", ha="left", va="center")
    ax.text(0.15, 1.05, "ε ∈ (-1, 0)\ninelástica", ha="left", va="center")
    # títulos e rótulos em português
    ax.set_title("Exemplo de curva de demanda P = 4 − 2Q\ncom elasticidade-preço da quantidade ε = 1 − 2/Q")
    ax.set_xlabel("Quantidade Q")
    ax.set_ylabel("Preço P")
    # ajusta limites do gráfico
    ax.set_xlim(0, 2.05)
    ax.set_ylim(0, 4.1)
    # fecha a figura para controle externo
    #plt.close(fig)
    # retorna a figura para exibição ou salvamento
    #return fig


###################################################################################################################
# gera a base transacional inicial apenas com ID, anomes e preço pago
def gerar_df_preco(n=200_000, x_min=0.0, x_max=1.0, A=80.0, B=-30.0, seed=42, anomes=202501):
    # fixa a semente aleatória para garantir reprodutibilidade
    np.random.seed(seed)
    # gera uma população potencial maior do que a observada
    x = np.random.uniform(x_min, x_max, n)
    # define o preço pago como função linear decrescente da captação latente
    valor_pago = A + B * x
    # adiciona ruído leve ao preço para evitar alinhamento perfeito
    valor_pago = valor_pago + np.random.normal(0, 0.5, n)
    # define uma probabilidade de compra decrescente no preço (quick and dirty)
    prob_compra = 1 - (valor_pago - valor_pago.min()) / (valor_pago.max() - valor_pago.min())
    # garante que a probabilidade esteja no intervalo [0, 1]
    prob_compra = np.clip(prob_compra, 0.01, 0.99)
    # sorteia quais indivíduos efetivamente compram
    compra = np.random.binomial(1, prob_compra)
    # filtra apenas os indivíduos que realizaram compra
    valor_pago = valor_pago[compra == 1]
    # constrói a base transacional inicial apenas com compradores
    df = pd.DataFrame({"ID": np.arange(1, len(valor_pago) + 1), "Anomes": anomes, "ValorPago": valor_pago})
    # retorna o DataFrame inicial pronto para enriquecimento
    return df

# Funções da esteira analítica
###################################################################################################################
# acrescenta elasticidade individual e valor futuro à base transacional existente
def adicionar_elasticidade_e_valor_futuro(df, a=1.2, b=0.015, c=0.00012, noise_std=0.05, seed=42):
    # fixa a semente aleatória para garantir reprodutibilidade
    np.random.seed(seed)
    # extrai o preço pago como array numpy
    valor_pago = df["ValorPago"].values
    # define a elasticidade individual como função quadrática do preço com ruído leve
    elasticidade = a + b * valor_pago - c * valor_pago**2 + np.random.normal(0, noise_std, len(valor_pago))
    # impõe piso mínimo para evitar elasticidades negativas
    elasticidade = np.clip(elasticidade, 0.01, None)
    # calcula o valor futuro como produto direto entre elasticidade e preço pago
    valor_futuro = elasticidade * valor_pago
    # cria uma cópia do DataFrame para evitar efeitos colaterais
    df = df.copy()
    # adiciona a coluna de valor futuro
    df["ValorFuturo"] = valor_futuro
    # adiciona a elasticidade observada como razão entre valor futuro e valor pago
    df["Elasticidade"] = df["ValorFuturo"] / df["ValorPago"]
    # retorna o DataFrame final enriquecido
    return df

#
def agregar_por_elasticidade(df, n_bins=20):
    # cria limites dos buckets como faixas uniformes no intervalo observado da elasticidade
    bins = np.linspace(df["Elasticidade"].min(), df["Elasticidade"].max(), n_bins + 1)
    # cria uma cópia do DataFrame para evitar efeitos colaterais
    df = df.copy()
    # atribui cada observação a um bucket de elasticidade baseado em range contínuo
    df["bin_elasticidade"] = pd.cut(df["Elasticidade"], bins=bins, include_lowest=True)
    # obtém a ordem numérica dos bins (0, 1, 2, ...) para nomeação estável
    df["_bin_idx"] = df["bin_elasticidade"].cat.codes
    # cria um nome ordenável alfabeticamente para cada bucket (bin_01, bin_02, ...)
    df["bucket_nome"] = df["_bin_idx"].apply(lambda x: f"bin_{x+1:02d}")
    # agrega os dados por bucket calculando elasticidade mediana, mediana do preço pago e captação
    agg = df.groupby("bin_elasticidade", as_index=False
                    ).agg(Elasticidade_media=("Elasticidade", "median"), 
                          ValorPago_media=("ValorPago", "median"), 
                          Captacao=("ID", "nunique"))
    # recria o índice numérico do bin na tabela agregada para manter consistência de nomes
    agg["_bin_idx"] = agg["bin_elasticidade"].cat.codes
    # atribui o nome ordenável do bucket também na tabela agregada
    agg["bucket_nome"] = agg["_bin_idx"].apply(lambda x: f"bin_{x+1:02d}")
    # remove a coluna auxiliar de índice do bin
    agg = agg.drop(columns="_bin_idx")
    # retorna a tabela agregada pronta para análise e plotagem da curva tipo Laffer
    return agg


#
def plot_laffer_scatter(agg):
    # extrai a elasticidade média de cada bucket para o eixo x
    x = agg["Elasticidade_media"].values
    # extrai a captação (quantidade de pagantes) de cada bucket para o eixo y
    y = agg["Captacao"].values
    # cria explicitamente a figura e o eixo para controle do objeto retornado
    fig, ax = plt.subplots(figsize=(8, 5))
    # plota o scatter de captação versus elasticidade no eixo criado
    ax.scatter(x, y)
    # define o rótulo do eixo x como elasticidade média observada
    ax.set_xlabel("Elasticidade média (ValorFuturo / ValorPago)")
    # define o rótulo do eixo y como captação (contagem de pagantes)
    ax.set_ylabel("Captação (quantidade de pagantes)")
    # define o título descrevendo explicitamente o objeto da curva
    ax.set_title("Curva tipo Laffer: Captação vs Elasticidade")
    # fecha a figura para evitar renderização automática fora do controle da função
    plt.close(fig)
    # retorna o objeto figura para uso posterior (exibição, salvamento ou composição)
    return fig

#
def plot_laffer_quadratico(agg):
    # extrai a elasticidade média de cada bucket para o eixo x
    x = agg["Elasticidade_media"].values
    # extrai a captação (quantidade de pagantes) de cada bucket para o eixo y
    y = agg["Captacao"].values
    # ajusta uma parábola de grau 2 (aproximação quick and dirty da curva de Laffer)
    coef = np.polyfit(x, y, 2)
    # constrói o polinômio correspondente aos coeficientes ajustados
    poly = np.poly1d(coef)
    # cria uma grade contínua de elasticidade para desenhar a curva ajustada
    x_fit = np.linspace(x.min(), x.max(), 200)
    # avalia o polinômio ajustado sobre a grade contínua
    y_fit = poly(x_fit)
    # cria explicitamente a figura e o eixo para controle do objeto retornado
    fig, ax = plt.subplots(figsize=(8, 5))
    # plota o scatter original de captação versus elasticidade
    ax.scatter(x, y)
    # plota a curva quadrática ajustada sobre os pontos observados
    ax.plot(x_fit, y_fit)
    # define o rótulo do eixo x como elasticidade média
    ax.set_xlabel("Elasticidade média")
    # define o rótulo do eixo y como captação (quantidade de pagantes)
    ax.set_ylabel("Captação")
    # define o título indicando explicitamente o tipo de aproximação usada
    ax.set_title("Aproximação quadrática da curva de captação")
    # fecha a figura para evitar renderização automática fora da função
    plt.close(fig)
    # retorna a figura e os coeficientes da parábola (a, b, c)
    return fig, coef

# plota a curva tipo Laffer com ajuste quadrático e calcula o R² do fit
def plot_laffer_quadratico(agg):
    # extrai a elasticidade média de cada bucket para o eixo x
    x = agg["Elasticidade_media"].values
    # extrai a captação (quantidade de pagantes) de cada bucket para o eixo y
    y = agg["Captacao"].values
    # ajusta uma parábola de grau 2 (aproximação quick and dirty da curva de Laffer)
    coef = np.polyfit(x, y, 2)
    # constrói o polinômio correspondente aos coeficientes ajustados
    poly = np.poly1d(coef)
    # calcula os valores ajustados exatamente nos pontos observados
    y_hat = poly(x)
    # calcula o coeficiente de determinação R² do ajuste
    r2 = 1 - np.sum((y - y_hat) ** 2) / np.sum((y - y.mean()) ** 2)
    # cria uma grade contínua de elasticidade para desenhar a curva ajustada
    x_fit = np.linspace(x.min(), x.max(), 200)
    # avalia o polinômio ajustado sobre a grade contínua
    y_fit = poly(x_fit)
    # cria explicitamente a figura e o eixo para controle do objeto retornado
    fig, ax = plt.subplots(figsize=(8, 5))
    # plota o scatter original de captação versus elasticidade
    ax.scatter(x, y)
    # plota a curva quadrática ajustada sobre os pontos observados
    ax.plot(x_fit, y_fit)
    # define o rótulo do eixo x como elasticidade média
    ax.set_xlabel("Elasticidade média")
    # define o rótulo do eixo y como captação (quantidade de pagantes)
    ax.set_ylabel("Captação")
    # define o título indicando explicitamente o tipo de aproximação usada e o R²
    ax.set_title(f"Aproximação quadrática da curva de captação (R² = {r2:.3f})")
    # fecha a figura para evitar renderização automática fora da função
    plt.close(fig)
    # retorna a figura, os coeficientes da parábola e o R² do ajuste
    return fig, coef, r2

# agrega por preço com bins definidos por step fixo (range pré-definido), evitando bin de borda minúsculo
def agregar_por_preco_step(df, p_min=50.0, p_max=80.0, step=1.5):
    # cria os limites dos bins por passo fixo, com suporte controlado
    bins = np.arange(p_min, p_max + step, step)
    # cria uma cópia do DataFrame para evitar efeitos colaterais
    df = df.copy()
    # filtra o intervalo de preço para não poluir o primeiro/último bin com outliers de ruído
    df = df[(df["ValorPago"] >= p_min) & (df["ValorPago"] <= p_max)]
    # atribui cada observação a um bucket de preço baseado em ranges fixos
    df["bin_preco"] = pd.cut(df["ValorPago"], bins=bins, include_lowest=True)
    # obtém a ordem numérica dos bins para nomeação estável
    df["_bin_idx"] = df["bin_preco"].cat.codes
    # cria um nome ordenável alfabeticamente para cada bucket (bin_01, bin_02, ...)
    df["bucket_nome"] = df["_bin_idx"].apply(lambda x: f"bin_{x+1:02d}")
    # agrega os dados por bucket calculando preço mediano e captação
    agg_preco = df.groupby("bin_preco", as_index=False
                          ).agg(ValorPago_mediana=("ValorPago", "median"), 
                                Captacao=("ID", "nunique"))
    # recria o índice numérico do bin na tabela agregada para manter consistência de nomes
    agg_preco["_bin_idx"] = agg_preco["bin_preco"].cat.codes
    # atribui o nome ordenável do bucket também na tabela agregada
    agg_preco["bucket_nome"] = agg_preco["_bin_idx"].apply(lambda x: f"bin_{x+1:02d}")
    # remove a coluna auxiliar de índice do bin
    agg_preco = agg_preco.drop(columns="_bin_idx")
    # retorna a tabela agregada por preço pronta para regressão e plot
    return agg_preco


# plota a relação Captação (x) versus Preço mediano (y) a partir do agregado por preço
def plot_preco_vs_captacao(agg_preco):
    # extrai a captação por bucket para o eixo x
    x = agg_preco["Captacao"].values
    # extrai o preço mediano por bucket para o eixo y
    y = agg_preco["ValorPago_mediana"].values
    # cria explicitamente a figura e o eixo
    fig, ax = plt.subplots(figsize=(8, 5))
    # plota o scatter captação versus preço
    ax.scatter(x, y)
    # define rótulos e título
    ax.set_xlabel("Captação (quantidade)")
    ax.set_ylabel("Preço pago mediano")
    ax.set_title("Captação vs Preço pago (bins por step fixo)")
    # fecha a figura para controle externo
    plt.close(fig)
    # retorna a figura
    return fig


# ajusta uma regressão linear P = A + B x entre Captação (x) e Preço médio (y)
def regressao_linear_preco_captacao(agg_preco):
    # extrai a variável explicativa como captação observada
    x = agg_preco["Captacao"].values
    # extrai a variável resposta como preço pago mediano
    y = agg_preco["ValorPago_mediana"].values
    # constrói a matriz de desenho com termo constante
    X = np.column_stack([np.ones(len(x)), x])
    # estima os coeficientes via mínimos quadrados ordinários
    coef = np.linalg.lstsq(X, y, rcond=None)[0]
    # separa intercepto e inclinação da reta
    A, B = coef
    # calcula os valores ajustados pela regressão
    y_hat = X @ coef
    # calcula o coeficiente de determinação R²
    r2 = 1 - np.sum((y - y_hat)**2) / np.sum((y - y.mean())**2)
    # cria explicitamente a figura e o eixo para visualização controlada
    fig, ax = plt.subplots(figsize=(8, 5))
    # plota os pontos observados de captação versus preço
    ax.scatter(x, y)
    # plota a reta ajustada da regressão linear
    ax.plot(x, y_hat)
    # define o rótulo do eixo x como captação (demanda)
    ax.set_xlabel("Captação (demanda)")
    # define o rótulo do eixo y como preço pago mediano
    ax.set_ylabel("Preço pago mediano")
    # define o título explicitando a forma funcional estimada
    ax.set_title("Regressão linear: Preço = A + B · Captação")
    # fecha a figura para evitar renderização automática fora da função
    plt.close(fig)
    # retorna a figura, os coeficientes da reta e o R²
    return fig, A, B, r2

# Introdução

In [ ]:
plot_curva_demanda_elasticidade_exemplo()

# Esteira analítica

## Gera o dataset sintético

In [ ]:
df_preco = gerar_df_preco(n=200_000, x_min=0.0, x_max=1.0, A=80.0, B=-30.0, seed=42, anomes=202501)

## Cria a curva de elasticidade teórica

In [ ]:
df = adicionar_elasticidade_e_valor_futuro(df_preco, a=1.2, b=0.015, c=0.00012, noise_std=0.05, seed=42)

## Agrega por faixa de preço (binning)

In [ ]:
#
agg_preco = agregar_por_preco_step(df, p_min=50.0, p_max=80.0, step=1.5)

## Plot da curva de preço vs demanda

In [ ]:
fig = plot_preco_vs_captacao(agg_preco)

## Regressão linear para encontrar a curva de preço

In [ ]:
# ajusta a regressão linear entre preço e captação
fig_reg, A_hat, B_hat, r2 = regressao_linear_preco_captacao(agg_preco)
# exibe a figura com a reta ajustada
display(fig_reg)
# imprime os coeficientes estimados e a qualidade do ajuste
print(f"A = {A_hat:.3f}, B = {B_hat:.3f}, R² = {r2:.3f}")

## Agrega por intervalo de elasticidade (binning)

In [ ]:
# agrega os dados em buckets por range de elasticidade para permitir variação real de captação
agg_elast = agregar_por_elasticidade(df, n_bins=20)

## Plot da curva de Laffer

In [ ]:
# plota o scatter Elasticidade média (x) versus Captação (y), retornando a figura fechada
fig_laffer = plot_laffer_scatter(agg_elast)

## Fit polinomial da curva teórica de Laffer

In [ ]:
# ajusta a aproximação quadrática da curva de captação e retorna a figura e os coeficientes
fig_quad, coeficientes, r2 = plot_laffer_quadratico(agg_elast)
# imprime os coeficientes da parábola ajustada (a, b, c) para inspeção rápida
print("a, b, c:"); print(coeficientes) ; print('r2:', r2)
fig_quad